In [1]:
# %load transformer/evaluate.py
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
from tqdm import *
import numpy as np
from transformer.models import TrfmSeq2seqProp2
from transformer.dataset import Seq2seqDatasetProp
from transformer.build_vocab import WordVocab
import json
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sys
import joblib

torch.manual_seed(0)

params = json.load(open("exp_params/params_4.json"))
model_file = "exps/exp4/ST_48_1767-0.003617.pkl"
assert torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

vocab = WordVocab.load_vocab(params['vocab'])
model = TrfmSeq2seqProp2(len(vocab), params['hidden'], len(vocab), params['n_layer'])
model.load_state_dict(torch.load(model_file))
model = nn.DataParallel(model)
model.to(device)
model.eval()

# Load the scaler
scaler = joblib.load("data/scaler.pkl")


In [2]:
batch_size = params['batch_size']
dataset = Seq2seqDatasetProp(params, vocab)

_, test = torch.utils.data.random_split(
                dataset, [len(dataset) - params['test_size'], params['test_size']])

test_loader = DataLoader(test, batch_size=batch_size, shuffle=False, num_workers=56)


In [28]:
ytrue = np.empty((len(test), 3), np.float32)
ypred = np.empty_like(ytrue, np.float32)

print(f"Evaluating {params['data']}...")
with torch.no_grad():
    for i, (x, y) in enumerate(tqdm(test_loader)):
        _, pred = model(x.to(device))
        ytrue[i*batch_size:i*batch_size+batch_size] = y.numpy()
        ypred[i*batch_size:i*batch_size+batch_size] = pred.cpu().numpy()
        break

  0%|          | 0/10 [00:00<?, ?it/s]

Evaluating data/chembl_24_chemreps_processed_scaled.csv...


  0%|          | 0/10 [00:13<?, ?it/s]


In [27]:
y.device, pred.device

(device(type='cpu'), device(type='cuda', index=0))

In [29]:
ypred

array([[ 3.6969191e-01,  7.0472121e-02,  5.9174597e-01],
       [ 3.6320645e-01,  5.7177496e-01,  2.5192970e-01],
       [ 3.6612374e-01,  7.6592195e-01,  1.7208233e-01],
       ...,
       [ 4.5675323e-41, -6.5101274e+35,  4.5675323e-41],
       [-3.5327679e+35,  4.5675323e-41, -5.4986278e+35],
       [ 4.5675323e-41, -6.5101559e+35,  4.5675323e-41]], dtype=float32)

In [5]:
# #"logP", "qed", "sas"
# # Inverse transform
# ytrue = scaler.inverse_transform(ytrue)
# ypred = scaler.inverse_transform(ypred)
# mae = [mean_absolute_error(ytrue[:, i], ypred[:, i]) for i in range(3)]
# rmse = [np.sqrt(mean_squared_error(ytrue[:, i], ypred[:, i])) for i in range(3)]

In [6]:
ytrue[:10], ypred[:10]

(array([[5.6402025 , 0.06705579, 6.270664  ],
        [5.6402025 , 0.06705579, 6.270664  ],
        [5.6402025 , 0.06705579, 6.270664  ],
        [5.6402025 , 0.06705579, 6.270664  ],
        [5.6402025 , 0.06705579, 6.270664  ],
        [5.6402025 , 0.06705579, 6.270664  ],
        [5.6402025 , 0.06705579, 6.270664  ],
        [5.6402025 , 0.06705579, 6.270664  ],
        [5.6402025 , 0.06705579, 6.270664  ],
        [5.6402025 , 0.06705579, 6.270664  ]], dtype=float32),
 array([[4.842292  , 0.06820208, 6.3257136 ],
        [4.842292  , 0.06820208, 6.3257136 ],
        [4.842292  , 0.06820208, 6.3257136 ],
        [4.842292  , 0.06820208, 6.3257136 ],
        [4.842292  , 0.06820208, 6.3257136 ],
        [4.842292  , 0.06820208, 6.3257136 ],
        [4.842292  , 0.06820208, 6.3257136 ],
        [4.842292  , 0.06820208, 6.3257136 ],
        [4.842292  , 0.06820208, 6.3257136 ],
        [4.842292  , 0.06820208, 6.3257136 ]], dtype=float32))

In [14]:
scaler.inverse_transform(y)

array([[5.64020241, 0.06705579, 6.2706641 ],
       [2.23259779, 0.62665498, 2.25496657],
       [3.80459593, 0.75104895, 2.0205635 ],
       ...,
       [3.1105973 , 0.24076508, 2.98288631],
       [5.05849649, 0.35952767, 4.03036109],
       [0.53500028, 0.50814179, 4.21724641]])

In [15]:
y

tensor([[0.3726, 0.0693, 0.5856],
        [0.3603, 0.6603, 0.1394],
        [0.3660, 0.7916, 0.1134],
        ...,
        [0.3635, 0.2527, 0.2203],
        [0.3705, 0.3781, 0.3367],
        [0.3542, 0.5351, 0.3575]])

In [17]:
y = scaler.inverse_transform(y); pred = scaler.inverse_transform(pred)
y_, p_ = y.cpu().numpy(), pred.cpu().numpy()

AttributeError: 'numpy.ndarray' object has no attribute 'cpu'

In [13]:
mean_absolute_error(y_[:, 0], p_[:, 0])

0.00319019

In [24]:
ytrue = np.empty((len(test), 3), np.float32)
ypred = np.empty_like(ytrue, np.float32)

print(f"Evaluating {params['data']}...")
with torch.no_grad():
    for i, (x, y) in enumerate(tqdm(test_loader)):
        _, pred = model(x.to(device))
        print(y)
        print(pred)
        break
        for p, q in zip(y, pred):
            ytrue[i*batch_size:i*batch_size+batch_size] = p.cpu().numpy()
            ypred[i*batch_size:i*batch_size+batch_size] = q.cpu().numpy()

#"logP", "qed", "sas"
# Inverse transform
ytrue = scaler.inverse_transform(ytrue)
ypred = scaler.inverse_transform(ypred)
mae = [mean_absolute_error(ytrue[:, i], ypred[:, i]) for i in range(3)]
rmse = [np.sqrt(mean_squared_error(ytrue[:, i], ypred[:, i])) for i in range(3)]


  0%|          | 0/10 [00:00<?, ?it/s]

Evaluating data/chembl_24_chemreps_processed_scaled.csv...
tensor([[0.3726, 0.0693, 0.5856],
        [0.3603, 0.6603, 0.1394],
        [0.3660, 0.7916, 0.1134],
        ...,
        [0.3635, 0.2527, 0.2203],
        [0.3705, 0.3781, 0.3367],
        [0.3542, 0.5351, 0.3575]])
tensor([[0.3697, 0.0705, 0.5917],
        [0.3632, 0.5718, 0.2519],
        [0.3661, 0.7659, 0.1721],
        ...,
        [0.3738, 0.1974, 0.2007],
        [0.3609, 0.3446, 0.2735],
        [0.3590, 0.4156, 0.3972]], device='cuda:0')


  0%|          | 0/10 [00:13<?, ?it/s]
/home/mhassan/miniconda3/envs/smiles_transformer/lib/python3.6/site-packages/sklearn/preprocessing/_data.py:437: RuntimeWarning: overflow encountered in true_divide
  X /= self.scale_
/home/mhassan/miniconda3/envs/smiles_transformer/lib/python3.6/site-packages/sklearn/preprocessing/_data.py:437: RuntimeWarning: overflow encountered in true_divide
  X /= self.scale_


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:

def get_metrics():
    # Evaluate test data
    batch_size = params['batch_size']
    dataset = Seq2seqDatasetProp(params, vocab)

    _, test = torch.utils.data.random_split(
                    dataset, [len(dataset) - params['test_size'], params['test_size']])
    
    test_loader = DataLoader(test, batch_size=batch_size, shuffle=False, num_workers=56)
    ytrue = np.empty((len(test), 3), np.float32)
    ypred = np.empty_like(ytrue, np.float32)
    
    print(f"Evaluating {params['data']}...")
    with torch.no_grad():
        for i, (x, y) in enumerate(tqdm(test_loader)):
            _, pred = model(x.to(device))
            for p, q in zip(y, pred):
                ytrue[i*batch_size:i*batch_size+batch_size] = p.cpu().numpy()
                ypred[i*batch_size:i*batch_size+batch_size] = q.cpu().numpy()
    
    #"logP", "qed", "sas"
    # Inverse transform
    ytrue = scaler.inverse_transform(ytrue)
    ypred = scaler.inverse_transform(ypred)
    mae = [mean_absolute_error(ytrue[:, i], ypred[:, i]) for i in range(3)]
    rmse = [np.sqrt(mean_squared_error(ytrue[:, i], ypred[:, i])) for i in range(3)]

    print(f"VALUES: {params['props']}")
    print(f"MEAN ABSOLUTE ERROR: {mae}")
    print(f"ROOT MEAN SQUARED ERROR: {rmse}")

    print("--------------------------------")

    # Evaluate zinc data
    params['data'] = TESTDATA # replace the training data file with the test data file
    dataset = Seq2seqDatasetProp(params, vocab)
    test_loader = DataLoader(dataset, batch_size = batch_size, num_workers=56, shuffle=False)
    ytrue = np.empty((len(dataset), 3), np.float32)
    ypred = np.empty_like(ytrue, np.float32)

    print(f"Evaluating {params['data']}...")
    with torch.no_grad():
        for i, (x, y) in enumerate(tqdm(test_loader)):
            _, pred = model(x.to(device))
            for p, q in zip(y, pred):
                ytrue[i*batch_size:i*batch_size+batch_size] = p.cpu().numpy()
                ypred[i*batch_size:i*batch_size+batch_size] = q.cpu().numpy()
    
    #"logP", "qed", "sas"
    # Inverse transform
    ytrue = scaler.inverse_transform(ytrue)
    ypred = scaler.inverse_transform(ypred)
    mae = [mean_absolute_error(ytrue[:, i], ypred[:, i]) for i in range(3)]
    rmse = [np.sqrt(mean_squared_error(ytrue[:, i], ypred[:, i])) for i in range(3)]
    print(f"VALUES: {params['props']}")
    print(f"MEAN ABSOLUTE ERROR: {mae}")
    print(f"ROOT MEAN SQUARED ERROR: {rmse}")


def encode():
    output = np.empty((len(dataset), 1024), dtype=np.float32)
    with torch.no_grad():
        for i, (x, y) in enumerate(tqdm(test_loader)):
            _encoder = model.module._encode(x.to(device))
            output[i*batch_size:i*batch_size+batch_size] = _encoder
    np.save('encoded', output)
    print("Encoded representations are saved to encoded.npy")


if __name__=="__main__":
    get_metrics()
    #encode()
